In [10]:
#########
### importing bla bla bla ####
import pandas as pd
import numpy as np
import seaborn as sb
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)

from sklearn.cross_validation import train_test_split
from pandas import Series, DataFrame
from pylab import rcParams


from sklearn.metrics import classification_report,accuracy_score
sb.set(style="white")
sb.set(style="whitegrid", color_codes=True)
import itertools
from sklearn import tree
import graphviz
from sklearn.model_selection import GridSearchCV,StratifiedKFold, LeaveOneOut
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestClassifier

import warnings; warnings.simplefilter('ignore')






### done with the imports ###



In [11]:
### reading data and printing the data ###
data = pd.read_csv('classical_feature.csv',names=['ID','TruncatedID','Mean','SD','AUC','Peak','MaxIndex','Halfpk','Skew','HHMean','Fat','Carb','Protein'
])
# data = data.dropna()
print(data.shape)
print(list(data.columns))


### done with reading data and printing the data ###

(171, 13)
['ID', 'TruncatedID', 'Mean', 'SD', 'AUC', 'Peak', 'MaxIndex', 'Halfpk', 'Skew', 'HHMean', 'Fat', 'Carb', 'Protein']


In [12]:
### CHO
CHO_data = data.drop(['ID','Protein','Carb'], axis=1)
print(CHO_data.Fat.unique())
print(CHO_data.TruncatedID.unique())


[20.  18.   2.5]
['2133-002' '2133-022' '2133-020' '2133-004' '2133-010' '2133-015'
 '2133-040' '2133-013' '2133-008' '2133-019' '2133-001' '2133-024'
 '2133-011' '2133-018' '2133-006' '2133-032' '2133-025' '2133-017'
 '2133-028' '2133-039' '2133-021' '2133-030' '2133-033' '2133-037'
 '2133-026' '2133-012' '2133-036' '2133-035' '2133-041' '2133-009']


In [13]:
###for LOW-HIGH

CHO_LH_data = CHO_data[CHO_data['Fat'].isin([2.5,20.0])]
CHO_LH_data.loc[CHO_LH_data.Fat == 2.5, 'CHO_bin'] = 0
CHO_LH_data.loc[CHO_LH_data.Fat== 20.0, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Fat'], axis=1)

X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin']
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin']

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009']
final_accuracy =[]
for i in range(0,30):
    print("For fold :", i+1)
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])]
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]
    X_train = train.loc[:, train.columns != 'CHO_bin']
    Y_train = train.loc[:, train.columns == 'CHO_bin']
    X_test = test.loc[:, test.columns != 'CHO_bin']
    
    Y_test =test.loc[:, test.columns == 'CHO_bin']
    train_ids = X_train['TruncatedID']
  
    train_ids = np.array(train_ids)
    train_ids = np.unique(train_ids)
#     print(train_ids[1])
    X_train = X_train.drop(['TruncatedID'],axis = 1)
    X_test = X_test.drop(['TruncatedID'],axis = 1)
    max_acc = 0
    for k in range(1,6):
        ypred_all = []
        for j in range(0,29):
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])]
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin']
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']
            xtr = xtr.drop(['TruncatedID'],axis = 1)
            xvl = xvl.drop(['TruncatedID'],axis = 1)
            rf = tree.DecisionTreeClassifier(max_depth=k)
            rf = rf.fit(xtr,ytr)
            ypred = rf.predict(xvl)
            ypred_all.append(accuracy_score(yvl,ypred))
#         ypred_all = np.array(ypred_all)
        ypred_mean = np.mean(ypred_all)
#         print(k,ypred_mean)
        if max_acc<ypred_mean:
            max_acc = ypred_mean
            best_param = k
#     print(max_acc, best_param)
    rf = tree.DecisionTreeClassifier(max_depth=best_param)
    rf = rf.fit(X_train,Y_train)
    Y_pred = rf.predict(X_test)
#     print(classification_report(Y_test,Y_pred))
#     print('Accuracy for test set :',accuracy_score(Y_test,Y_pred))
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
print("The final accuracy is : ", np.mean(final_accuracy))
            
    

For fold : 1
For fold : 2
For fold : 3
For fold : 4
For fold : 5
For fold : 6
For fold : 7
For fold : 8
For fold : 9
For fold : 10
For fold : 11
For fold : 12
For fold : 13
For fold : 14
For fold : 15
For fold : 16
For fold : 17
For fold : 18
For fold : 19
For fold : 20
For fold : 21
For fold : 22
For fold : 23
For fold : 24
For fold : 25
For fold : 26
For fold : 27
For fold : 28
For fold : 29
For fold : 30
The final accuracy is :  0.75


In [14]:
###for MEDIUM-HIGH

CHO_LH_data = CHO_data[CHO_data['Fat'].isin([18.0,20.0])]
CHO_LH_data.loc[CHO_LH_data.Fat == 18.0, 'CHO_bin'] = 0
CHO_LH_data.loc[CHO_LH_data.Fat== 20.0, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Fat'], axis=1)

X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin']
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin']

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009']
final_accuracy =[]
for i in range(0,30):
    print("For fold :", i+1)
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])]
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]
    X_train = train.loc[:, train.columns != 'CHO_bin']
    Y_train = train.loc[:, train.columns == 'CHO_bin']
    X_test = test.loc[:, test.columns != 'CHO_bin']
    
    Y_test =test.loc[:, test.columns == 'CHO_bin']
    train_ids = X_train['TruncatedID']
  
    train_ids = np.array(train_ids)
    train_ids = np.unique(train_ids)
#     print(train_ids[1])
    X_train = X_train.drop(['TruncatedID'],axis = 1)
    X_test = X_test.drop(['TruncatedID'],axis = 1)
    max_acc = 0
    for k in range(1,6):
        ypred_all = []
        for j in range(0,29):
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])]
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin']
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']
            xtr = xtr.drop(['TruncatedID'],axis = 1)
            xvl = xvl.drop(['TruncatedID'],axis = 1)
            rf = tree.DecisionTreeClassifier(max_depth=k)
            rf = rf.fit(xtr,ytr)
            ypred = rf.predict(xvl)
            ypred_all.append(accuracy_score(yvl,ypred))
#         ypred_all = np.array(ypred_all)
        ypred_mean = np.mean(ypred_all)
#         print(k,ypred_mean)
        if max_acc<ypred_mean:
            max_acc = ypred_mean
            best_param = k
#     print(max_acc, best_param)
    rf = tree.DecisionTreeClassifier(max_depth=best_param)
    rf = rf.fit(X_train,Y_train)
    Y_pred = rf.predict(X_test)
#     print(classification_report(Y_test,Y_pred))
#     print('Accuracy for test set :',accuracy_score(Y_test,Y_pred))
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
print("The final accuracy is : ", np.mean(final_accuracy))
            
    

For fold : 1
For fold : 2
For fold : 3
For fold : 4
For fold : 5
For fold : 6
For fold : 7
For fold : 8
For fold : 9
For fold : 10
For fold : 11
For fold : 12
For fold : 13
For fold : 14
For fold : 15
For fold : 16
For fold : 17
For fold : 18
For fold : 19
For fold : 20
For fold : 21
For fold : 22
For fold : 23
For fold : 24
For fold : 25
For fold : 26
For fold : 27
For fold : 28
For fold : 29
For fold : 30
The final accuracy is :  0.7166666666666667


In [15]:
###for LOW-Medium

CHO_LH_data = CHO_data[CHO_data['Fat'].isin([2.5,18.0])]
CHO_LH_data.loc[CHO_LH_data.Fat == 2.5, 'CHO_bin'] = 0
CHO_LH_data.loc[CHO_LH_data.Fat== 18.0, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Fat'], axis=1)

X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin']
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin']

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009']
final_accuracy =[]
for i in range(0,30):
    print("For fold :", i+1)
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])]
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]
    X_train = train.loc[:, train.columns != 'CHO_bin']
    Y_train = train.loc[:, train.columns == 'CHO_bin']
    X_test = test.loc[:, test.columns != 'CHO_bin']
    
    Y_test =test.loc[:, test.columns == 'CHO_bin']
    train_ids = X_train['TruncatedID']
  
    train_ids = np.array(train_ids)
    train_ids = np.unique(train_ids)
#     print(train_ids[1])
    X_train = X_train.drop(['TruncatedID'],axis = 1)
    X_test = X_test.drop(['TruncatedID'],axis = 1)
    max_acc = 0
    for k in range(1,6):
        ypred_all = []
        for j in range(0,29):
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])]
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin']
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']
            xtr = xtr.drop(['TruncatedID'],axis = 1)
            xvl = xvl.drop(['TruncatedID'],axis = 1)
            rf = tree.DecisionTreeClassifier(max_depth=k)
            rf = rf.fit(xtr,ytr)
            ypred = rf.predict(xvl)
            ypred_all.append(accuracy_score(yvl,ypred))
#         ypred_all = np.array(ypred_all)
        ypred_mean = np.mean(ypred_all)
#         print(k,ypred_mean)
        if max_acc<ypred_mean:
            max_acc = ypred_mean
            best_param = k
#     print(max_acc, best_param)
    rf = tree.DecisionTreeClassifier(max_depth=best_param)
    rf = rf.fit(X_train,Y_train)
    Y_pred = rf.predict(X_test)
#     print(classification_report(Y_test,Y_pred))
#     print('Accuracy for test set :',accuracy_score(Y_test,Y_pred))
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
print("The final accuracy is : ", np.mean(final_accuracy))
            
    

For fold : 1
For fold : 2
For fold : 3
For fold : 4
For fold : 5
For fold : 6
For fold : 7
For fold : 8
For fold : 9
For fold : 10
For fold : 11
For fold : 12
For fold : 13
For fold : 14
For fold : 15
For fold : 16
For fold : 17
For fold : 18
For fold : 19
For fold : 20
For fold : 21
For fold : 22
For fold : 23
For fold : 24
For fold : 25
For fold : 26
For fold : 27
For fold : 28
For fold : 29
For fold : 30
The final accuracy is :  0.5833333333333334
